# SE DER TEMPO:
- PADRONIZAR DADOS DE BENEFICIÁRIOS PARA 2021 SÓ PRA ESTADOS NECESSÁRIOS!!
- ANALISAR REDE DE SAÚDE DE SP E RJ
- DEFINIR NOME DA MÉTRICA proporcao_bh_apoio

# **Install and import of necessary libraries:**

In [ ]:
# installing necessary libraries that are not already installed by default on Google Colab:

# library to read .ods files:
!pip install odfpy;
print('')
print('===============')
print('')

# library to read datasets directly from Google Big Query:
!pip install basedosdados;
#!pip install google-resumable-media
#!pip install --upgrade googleapis-common-protos
print('')
print('===============')
print('')

# library to make more advanced data visualizations:
!pip install plotly_express;

In [ ]:
# importing libraries:

# basic libraries:
import numpy as np
import pandas as pd

# data import libraries:
import basedosdados as bd
from google.colab import drive
import glob

# data visualization libraries
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import plotly_express as px # bubble chart

# **Initial setup:**

In [ ]:
# connecting to my Google Drive:
drive.mount('/gdrive')

drive_path = '/gdrive/.shortcut-targets-by-id/1MNI7W_hrAxnm_EKmmeJI9LKcTUIegu2U/ASN.ROCKS'

Mounted at /gdrive


# **Declaration of functions:**

In [ ]:
def importa_arquivo(file_path, tipo_atendimento):

    df = pd.read_csv(file_path, sep=';')
    print(df.shape)

    df = df[df['CD_MUNICIPIO_BENEFICIARIO'].notna()]
    df['CD_MUNICIPIO_BENEFICIARIO'] = (df['CD_MUNICIPIO_BENEFICIARIO'].astype(int)).astype(str)

    df = df[df['CD_MUNICIPIO_PRESTADOR'].notna()]
    df['CD_MUNICIPIO_PRESTADOR'] = (df['CD_MUNICIPIO_PRESTADOR'].astype(int)).astype(str)

    df['AMB_OU_INTERN'] = tipo_atendimento
    print(df.shape)
    print('')

    return df

def gera_df_por_municipio(file_path, tipo_atendimento):

    print('')
    print(file_path)
    df = importa_arquivo(file_path, tipo_atendimento)

    if tipo_atendimento == 'AMBULATORIAL':
        coluna_data = 'DT_REALIZACAO'
    elif tipo_atendimento == 'INTERNACAO':
        coluna_data = 'ANO_MES_EVENTO'

    df_amb_por_municipio = df.groupby(by=['AMB_OU_INTERN', coluna_data, 'UF_PRESTADOR', 'CD_MUNICIPIO_PRESTADOR', 'CD_MUNICIPIO_BENEFICIARIO'], as_index=False).agg({
        'ID_EVENTO_ATENCAO_SAUDE': ['nunique']
    })
    df_amb_por_municipio.columns = df_amb_por_municipio.columns.droplevel(0)
    df_amb_por_municipio.columns = ['tipo_atendimento', 'mes', 'uf_prestador', 'id_municipio_prestador', 'id_municipio_beneficiario', 'qtde_atendimentos']

    print('')
    print(df_amb_por_municipio.head())
    print('')

    return df_amb_por_municipio

def junta_todos_os_meses(drive_path, tipo_atendimento):

    all_files = glob.glob(f'{drive_path}/PRODUCAO MEDICA ANS/2019/{tipo_atendimento}/*_2019*_*.csv')
    print(all_files)

    df_total = pd.concat((gera_df_por_municipio(file_path, tipo_atendimento) for file_path in all_files))

    return df_total



# **Begining of the code:**

In [ ]:
tipo_atendimento = 'AMBULATORIAL'
df_amb_por_municipio = junta_todos_os_meses(drive_path, tipo_atendimento)

In [ ]:
df_amb_por_municipio.head()

,tipo_atendimento,mes,uf_prestador,id_municipio_prestador,id_municipio_beneficiario,qtde_atendimentos
0,AMBULATORIAL,2019-01,ES,320010,310620,1
1,AMBULATORIAL,2019-01,ES,320010,313820,1
2,AMBULATORIAL,2019-01,ES,320010,317010,4
3,AMBULATORIAL,2019-01,ES,320010,317020,1
4,AMBULATORIAL,2019-01,ES,320010,320010,680


In [ ]:
df_amb_por_municipio.groupby(by=['uf_prestador', 'mes'], as_index=False).agg({
    'id_municipio_prestador': ['count', 'nunique'],
    'id_municipio_beneficiario': ['nunique'],
    'qtde_atendimentos': ['sum', 'mean', 'median']
})

uf_prestador      mes id_municipio_prestador          \
                                          count nunique   
0            ES  2019-01                   5470      72   
1            ES  2019-02                   5663      72   
2            ES  2019-03                   5435      70   
3            ES  2019-04                   5685      69   
4            ES  2019-05                   5871      69   
5            ES  2019-06                   5238      70   
6            ES  2019-07                   5867      72   
7            ES  2019-08                   5750      71   
8            ES  2019-09                   5342      70   
9            ES  2019-10                   5747      71   
10           ES  2019-11                   5496      72   
11           ES  2019-12                   5350      70   
12           MG  2019-01                  28431     660   
13           MG  2019-02                  27728     637   
14           MG  2019-03                  27697     580   
15           MG  2019-04                  28970     578   
16           MG  2019-05                  29482     578   
17           MG  2019-06                  27917     567   
18           MG  2019-07                  29718     584   
19           MG  2019-08                  28822     584   
20           MG  2019-09                  28530     573   
21           MG  2019-10                  29483     583   
22           MG  2019-11                  28406     586   
23           MG  2019-12                  27313     581   

   id_municipio_beneficiario qtde_atendimentos                     
                     nunique               sum        mean median  
0                       1340           1331761  243.466362    4.0  
1                       1501           1362645  240.622462    4.0  
2                       1470           1144123  210.510212    3.0  
3                       1496           1417629  249.363061    4.0  
4                       1534           1641733  279.634304    4.0  
5                       1213           1203123  229.691294    3.5  
6                       1553           1432644  244.186808    4.0  
7                       1509           1351908  235.114435    3.0  
8                       1244           1285610  240.660801    4.0  
9                       1491           1422595  247.536976    4.0  
10                      1465           1199303  218.213792    3.0  
11                      1419           1049527  196.173271    3.0  
12                      2751           4663399  164.025149    3.0  
13                      2751           4597984  165.824582    3.0  
14                      2776           4543981  164.060404    3.0  
15                      2837           5153596  177.894235    3.0  
16                      2865           5354038  181.603623    3.0  
17                      2814           4478984  160.439302    3.0  
18                      2860           5050748  169.955852    3.0  
19                      2865           4855778  168.474707    3.0  
20                      2832           4770731  167.218051    3.0  
21                      2850           5167363  175.265848    3.0  
22                      2829           4601647  161.995600    3.0  
23                      2805           3970136  145.357009    3.0

In [ ]:
df_amb_match = df_amb_por_municipio.groupby(by=['uf_prestador', 'id_municipio_prestador', 'id_municipio_beneficiario'], as_index=False).agg({
    'mes': ['count', 'nunique'],
    'qtde_atendimentos': ['sum', 'mean', 'median']
})
df_amb_match.columns = df_amb_match.columns.droplevel(0)
df_amb_match.columns = [
    'uf_prestador', 'id_municipio_prestador', 'id_municipio_beneficiario',
    'qtde', 'qtde_meses', 'total_atendimentos', 'media_atendimentos', 'mediana_atendimentos'
]
df_amb_match

,uf_prestador,id_municipio_prestador,id_municipio_beneficiario,qtde,qtde_meses,total_atendimentos,media_atendimentos,mediana_atendimentos
0,ES,320010,110032,1,1,1,1.000000,1.0
1,ES,320010,211130,2,2,2,1.000000,1.0
2,ES,320010,261160,1,1,1,1.000000,1.0
3,ES,320010,291080,1,1,1,1.000000,1.0
4,ES,320010,292200,1,1,1,1.000000,1.0
...,...,...,...,...,...,...,...,...
77685,MG,317210,330330,1,1,6,6.000000,6.0
77686,MG,317210,330455,9,9,28,3.111111,3.0
77687,MG,317210,330540,2,2,2,1.000000,1.0
77688,MG,317210,354880,2,2,3,1.500000,1.5


In [ ]:
df_amb_match.query("uf_prestador == 'MG' and id_municipio_prestador	!= id_municipio_beneficiario").\
    sort_values(by=['total_atendimentos'], ascending=False).head()

,uf_prestador,id_municipio_prestador,id_municipio_beneficiario,qtde,qtde_meses,total_atendimentos,media_atendimentos,mediana_atendimentos
18050,MG,310620,311860,12,12,1406856,117238.000000,115370.0
28089,MG,311860,310620,12,12,1099538,91628.166667,89632.0
17921,MG,310620,310670,12,12,591937,49328.083333,48028.5
18464,MG,310620,315460,12,12,338732,28227.666667,27251.5
18498,MG,310620,315780,12,12,296107,24675.583333,24129.0


In [ ]:
tipo_atendimento = 'INTERNACAO'
df_inter_por_municipio = junta_todos_os_meses(drive_path, tipo_atendimento)

In [ ]:
df_inter_por_municipio.head()

,tipo_atendimento,mes,uf_prestador,id_municipio_prestador,id_municipio_beneficiario,qtde_atendimentos
0,INTERNACAO,2019-11,ES,320010,320010,6
1,INTERNACAO,2019-11,ES,320020,317070,1
2,INTERNACAO,2019-11,ES,320020,320020,7
3,INTERNACAO,2019-11,ES,320040,320040,2
4,INTERNACAO,2019-11,ES,320040,320120,1


In [ ]:
df_inter_por_municipio.shape

(43296, 6)

In [ ]:
df_inter_por_municipio['mes'].value_counts(dropna=False)

2019-01    3886
2019-07    3793
2019-10    3776
2019-05    3772
2019-04    3702
2019-08    3684
2019-03    3615
2019-02    3606
2019-11    3558
2019-06    3501
2019-12    3418
2019-09    2985
Name: mes, dtype: int64

In [ ]:
df_inter_por_municipio.groupby(by=['uf_prestador', 'mes'], as_index=False).agg({
    'id_municipio_prestador': ['count', 'nunique'],
    'id_municipio_beneficiario': ['nunique'],
    'qtde_atendimentos': ['sum', 'mean', 'median']
})

uf_prestador      mes id_municipio_prestador          \
                                          count nunique   
0            ES  2019-01                    798      34   
1            ES  2019-02                    702      34   
2            ES  2019-03                    707      29   
3            ES  2019-04                    738      30   
4            ES  2019-05                    755      31   
5            ES  2019-06                    688      28   
6            ES  2019-07                    718      31   
7            ES  2019-08                    703      28   
8            ES  2019-10                    708      28   
9            ES  2019-11                    687      29   
10           ES  2019-12                    701      29   
11           MG  2019-01                   3088     195   
12           MG  2019-02                   2904     179   
13           MG  2019-03                   2908     171   
14           MG  2019-04                   2964     176   
15           MG  2019-05                   3017     179   
16           MG  2019-06                   2813     171   
17           MG  2019-07                   3075     181   
18           MG  2019-08                   2981     180   
19           MG  2019-09                   2985     175   
20           MG  2019-10                   3068     175   
21           MG  2019-11                   2871     170   
22           MG  2019-12                   2717     168   

   id_municipio_beneficiario qtde_atendimentos                    
                     nunique               sum       mean median  
0                        272             13261  16.617794      2  
1                        239             12752  18.165242      2  
2                        245             12691  17.950495      2  
3                        256             14134  19.151762      2  
4                        261             15237  20.181457      2  
5                        250             14128  20.534884      2  
6                        252             14887  20.733983      2  
7                        235             13724  19.522048      2  
8                        235             13916  19.655367      2  
9                        259             13453  19.582242      2  
10                       261             11503  16.409415      2  
11                      1018             47284  15.312176      1  
12                       984             44929  15.471419      1  
13                       994             43852  15.079780      1  
14                      1021             45777  15.444332      2  
15                      1034             50857  16.856811      2  
16                       978             45286  16.098827      2  
17                      1025             49502  16.098211      1  
18                      1011             47120  15.806776      1  
19                      1033             46698  15.644221      1  
20                      1033             47301  15.417536      1  
21                      1002             44067  15.349007      1  
22                       964             40968  15.078395      1

In [ ]:
df_inter_match = df_inter_por_municipio.groupby(by=['uf_prestador', 'id_municipio_prestador', 'id_municipio_beneficiario'], as_index=False).agg({
    'mes': ['count', 'nunique'],
    'qtde_atendimentos': ['sum', 'mean', 'median']
})
df_inter_match.columns = df_inter_match.columns.droplevel(0)
df_inter_match.columns = [
    'uf_prestador', 'id_municipio_prestador', 'id_municipio_beneficiario',
    'qtde', 'qtde_meses', 'total_atendimentos', 'media_atendimentos', 'mediana_atendimentos'
]
df_inter_match

,uf_prestador,id_municipio_prestador,id_municipio_beneficiario,qtde,qtde_meses,total_atendimentos,media_atendimentos,mediana_atendimentos
0,ES,320010,320010,9,9,41,4.555556,5.0
1,ES,320010,320030,1,1,1,1.000000,1.0
2,ES,320010,320130,1,1,1,1.000000,1.0
3,ES,320010,320500,1,1,1,1.000000,1.0
4,ES,320010,320506,1,1,1,1.000000,1.0
...,...,...,...,...,...,...,...,...
12711,MG,317200,316850,1,1,1,1.000000,1.0
12712,MG,317200,316990,1,1,1,1.000000,1.0
12713,MG,317200,317200,12,12,48,4.000000,4.0
12714,MG,317200,410690,1,1,1,1.000000,1.0


In [ ]:
df_inter_match.query("uf_prestador == 'MG' and id_municipio_prestador	!= id_municipio_beneficiario").\
    sort_values(by=['total_atendimentos'], ascending=False).head()

,uf_prestador,id_municipio_prestador,id_municipio_beneficiario,qtde,qtde_meses,total_atendimentos,media_atendimentos,mediana_atendimentos
3220,MG,310620,311860,12,12,16337,1361.416667,1359.5
8429,MG,314480,310620,12,12,13429,1119.083333,1123.5
3137,MG,310620,310670,12,12,6304,525.333333,519.0
4769,MG,311860,310620,12,12,4667,388.916667,393.0
3552,MG,310620,315780,12,12,3960,330.000000,297.0


In [ ]:
df_amb_por_municipio.to_csv('atendimentos_ambulatoriais.csv', index=False)
df_inter_por_municipio.to_csv('atendimentos_internacao.csv', index=False)